## 1. 환경 설정


In [20]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter # 문서의 단락을 청크 사이즈로 쪼개주는 역할
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

`pip install pymupdf`

In [22]:
# 1단계 : 문서 로드 
loader = PyMuPDFLoader("/Users/t2023-m0072/Desktop/assignment_LLM_RAG/RAG/PDF/인공지능산업최신동향_2024년11월호.pdf")
docs = loader.load()


In [23]:

# 확인
docs[2].page_content # Document 타입, 페이지 단위로 분할


'Ⅰ. 인공지능 산업 동향 브리프\n'

In [24]:

docs[2].metadata # 해당 페이지의 정보 확인


{'source': '/Users/t2023-m0072/Desktop/assignment_LLM_RAG/RAG/PDF/인공지능산업최신동향_2024년11월호.pdf',
 'file_path': '/Users/t2023-m0072/Desktop/assignment_LLM_RAG/RAG/PDF/인공지능산업최신동향_2024년11월호.pdf',
 'page': 2,
 'total_pages': 25,
 'format': 'PDF 1.4',
 'title': '',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'creator': 'Hwp 2018 10.0.0.13947',
 'producer': 'Hancom PDF 1.3.0.547',
 'creationDate': "D:20241105150426+09'00'",
 'modDate': "D:20241105150426+09'00'",
 'trapped': ''}

In [25]:

print(len(docs)) # 25 개 페이지 확인

25


In [26]:

# 2단계 : 문서 분할 (RecursiveCharacterTextSplitter)
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, # 각 청크 최대 100자 까지
    chunk_overlap=10, # 청크간 최대 10자 중복
    length_function=len, # 길이 계산 함수 : 문자 수 기준 (len())
    is_separator_regex=False, # 구분자가 정규 표현식인지 여부 (False : 정확히 일치하는 구분자 기준)
)

split_documents = recursive_text_splitter.split_documents(docs) # 청킹 결과물 저장

# 길이 확인
len(split_documents)

489

In [27]:
print(split_documents[1])

page_content='2024년 11월호
Ⅰ. 인공지능 산업 동향 브리프
 1. 정책/법제' metadata={'source': '/Users/t2023-m0072/Desktop/assignment_LLM_RAG/RAG/PDF/인공지능산업최신동향_2024년11월호.pdf', 'file_path': '/Users/t2023-m0072/Desktop/assignment_LLM_RAG/RAG/PDF/인공지능산업최신동향_2024년11월호.pdf', 'page': 1, 'total_pages': 25, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13947', 'producer': 'Hancom PDF 1.3.0.547', 'creationDate': "D:20241105150426+09'00'", 'modDate': "D:20241105150426+09'00'", 'trapped': ''}


In [28]:

# 청킹된 내용의 상위 10개 출력
top_10 = split_documents[:10]
# print(f"Chunk 3:{splits[3]}")
for i, chunk in enumerate(top_10, 1):
    page_content = chunk.page_content
    print(f"Chunk {i}:{page_content[:300]}\n\n")

Chunk 1:2024년 11월호


Chunk 2:2024년 11월호
Ⅰ. 인공지능 산업 동향 브리프
 1. 정책/법제


Chunk 3:▹ 미국 민권위원회, 연방정부의 얼굴인식 기술 사용에 따른 민권 영향 분석························ 1


Chunk 4:▹ 미국 백악관 예산관리국, 정부의 책임 있는 AI 조달을 위한 지침 발표····························· 2


Chunk 5:▹ 유로폴, 법 집행에서 AI의 이점과 과제를 다룬 보고서 발간·············································· 3


Chunk 6:▹ OECD, 공공 부문의 AI 도입을 위한 G7 툴킷 발표·························································· 4


Chunk 7:▹ 세계경제포럼, 생성AI 시대의 거버넌스 프레임워크 제시···················································· 5


Chunk 8:2. 기업/산업 
   ▹ CB인사이츠 분석 결과, 2024년 3분기 벤처 투자 31%가 AI 스타트업에 집중·············· 6


Chunk 9:▹ 메타, 동영상 생성AI 도구 ‘메타 무비 젠’


Chunk 10:‘메타 무비 젠’ 공개··································································· 7




In [29]:
# 3단계 : 벡터 임베딩 생성 
# 임베딩 : 청크 단위로 쪼개진 단락들을 숫자 표현으로 바꿔줌
embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002") # OpenAIEmbeddings 로 임베딩 알고리즘을 사용

In [30]:
# 4단계 : DB 생성 및 저장
vectorstore = FAISS.from_documents(
    documents=split_documents,  # 청킹된 결과물 (489 개로 분할된 문단)
    embedding=embeddings # 가져온 임베딩 알고리즘을 넣어줌
)  # split_documents 를 임베딩 알고리즘으로 임베딩하여 Vecor DB(FAISS) 에 저장

## 아직 PC 에 저장하진 않았고 메모리에 저장되어 있는 상태

In [32]:
# 5단계 : 검색기 (Retriever) 생성
retriever = vectorstore.as_retriever()
retriever # FAISS 라는 Vector DB 를 사용하는 Retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12a4009b0>, search_kwargs={})

In [33]:
# 검색기에 질문하기 - 비용 X
retriever.invoke("삼성전자가 개발한 생성AI 이름은 무엇인가요?") # 입력한 질문과 관련성이 높은 것부터 내림차순 정렬

[Document(metadata={'source': '/Users/t2023-m0072/Desktop/assignment_LLM_RAG/RAG/PDF/인공지능산업최신동향_2024년11월호.pdf', 'file_path': '/Users/t2023-m0072/Desktop/assignment_LLM_RAG/RAG/PDF/인공지능산업최신동향_2024년11월호.pdf', 'page': 13, 'total_pages': 25, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13947', 'producer': 'Hancom PDF 1.3.0.547', 'creationDate': "D:20241105150426+09'00'", 'modDate': "D:20241105150426+09'00'", 'trapped': ''}, page_content='∙카카오는 동 브랜드를 자사가 개발하는 주요 AI 모델과 신규 서비스의 이름에 두루 사용할 계획으로, \nAI 메이트 서비스 ‘카나나’ 출시 계획도 공개'),
 Document(metadata={'source': '/Users/t2023-m0072/Desktop/assignment_LLM_RAG/RAG/PDF/인공지능산업최신동향_2024년11월호.pdf', 'file_path': '/Users/t2023-m0072/Desktop/assignment_LLM_RAG/RAG/PDF/인공지능산업최신동향_2024년11월호.pdf', 'page': 15, 'total_pages': 25, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13947', 'producer': 'Hancom PDF 1.3.0.547', 'creationDate': "D

In [34]:
# 6단계 : 프롬프트 생성 
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Answer in Korean.
    
    #Question:
    {question}
     
    #Context:
    {context}
    
    #Answer:"""
)

# 7단계 : 언어모델(LLM) 생성
llm=ChatOpenAI(model_name='gpt-4o-mini',temperature=0)

# 8단계 : 체인(Chain) 생성
chain = (
    {'context':retriever, 'question': RunnablePassthrough()}
    | prompt # 위의 내용을 Prompt 로 전달
    | llm # llm 을 거쳐
    | StrOutputParser() # OutPut 이 나옴
)

In [35]:
# 체인 실행
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "카카오가 자체 개발한 AI 이름은?"
response = chain.invoke(question)
print(response)

카카오는 자체 개발한 AI 이름으로 '카나나'를 사용하고 있습니다.


In [ ]:
메타에서 자체 개발한 AI를 알아?